### Análise de Padrões em Acidentes Rodoviários no Brasil  2024 - Aprendizado de Máquina
___
#### **Objetivo**

Realizar o pré-processamento dos dados tratados na etapta anterior de exploração para convertê‑los em um formato transacional adequado, aplicar mineração de dados com regras de associação (algoritmo Apriori) para descobrir coocorrências significativas entre características dos acidentes e avalia essas regras para identificar padrões de risco.
___



#### Importação das bibliotecas 

In [2]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

#### Leitura do arquivo


In [3]:
acidentes_2024 = pd.read_csv('acidentes2024_todas_causas_tipos.csv', sep=';', encoding='latin-1') 
acidentes_2024.head()

,id,pesid,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_principal,...,sexo,ilesos,feridos_leves,feridos_graves,mortos,latitude,longitude,regional,delegacia,uop
0,571772.0,1268971.0,2024-01-01,segunda-feira,00:05:00,RJ,101.0,"272,5",TANGUA,Sim,...,Masculino,0.0,0.0,0.0,1.0,-22.729360,-42.701125,SPRF-RJ,DEL02-RJ,UOP03-DEL02-RJ
1,571774.0,1268985.0,2024-01-01,segunda-feira,00:05:00,GO,153.0,"424,6",ANAPOLIS,Não,...,Feminino,1.0,0.0,0.0,0.0,-16.229185,-49.009797,SPRF-GO,DEL02-GO,UOP01-DEL02-GO
2,571774.0,1268985.0,2024-01-01,segunda-feira,00:05:00,GO,153.0,"424,6",ANAPOLIS,Sim,...,Feminino,1.0,0.0,0.0,0.0,-16.229185,-49.009797,SPRF-GO,DEL02-GO,UOP01-DEL02-GO
3,571777.0,1269020.0,2024-01-01,segunda-feira,01:45:00,ES,101.0,"264,1",SERRA,Sim,...,Masculino,1.0,0.0,0.0,0.0,-20.172928,-40.267364,SPRF-ES,DEL02-ES,UOP01-DEL02-ES
4,571778.0,1269028.0,2024-01-01,segunda-feira,00:45:00,SC,101.0,110,PENHA,Não,...,Masculino,1.0,0.0,0.0,0.0,-26.834770,-48.706151,SPRF-SC,DEL03-SC,UOP02-DEL03-SC


In [4]:
# Criando um df para análises
df_acidentes = acidentes_2024.copy()

#### Regra de associação: Apriori

Regra de associação considerando:
- `Estado`
- `Tipo de pista`

In [5]:
valores_unicos_uf = df_acidentes['uf'].unique()
valores_unicos_tipo_pista = df_acidentes['tipo_pista'].unique()

print("Valores únicos na coluna 'uf':")
print(valores_unicos_uf)
print("\nValores únicos na coluna 'tipo_pista':")
print(valores_unicos_tipo_pista)

Valores únicos na coluna 'uf':
['RJ' 'GO' 'ES' 'SC' 'RS' 'PR' 'MS' 'RO' 'SP' 'MG' 'DF' 'PI' 'BA' 'PE'
 'SE' 'PB' 'TO' 'MT' 'CE' 'AL' 'RN' 'PA' 'AC' 'MA' 'RR' 'AP' 'AM']

Valores únicos na coluna 'tipo_pista':
['Dupla' 'Múltipla' 'Simples']


In [6]:
# Preparar as transações para UF e Tipo de Pista
transacoes_uf_pista = []
for index, row in df_acidentes.iterrows():
    transaction = []                   
    transaction.append(f"uf_{row['uf']}")  
    transaction.append(f"pista_{row['tipo_pista']}")
    transacoes_uf_pista.append(transaction)
    
# Converte para o formato adequado para o TransactionEncoder
uf_pista = TransactionEncoder()
array_uf_pista = uf_pista.fit(transacoes_uf_pista).transform(transacoes_uf_pista)
df_encoded_uf_pista = pd.DataFrame(array_uf_pista, columns=uf_pista.columns_)

In [7]:
# Aplicando Apriori com min_support 1% e a regra de associação
frequent_itemsets_uf_pista = apriori(df_encoded_uf_pista, min_support=0.01, use_colnames=True)
rules_uf_pista = association_rules(frequent_itemsets_uf_pista, metric="lift", min_threshold=1.0)

In [8]:
colunas_para_remover = ['representativity', 'leverage', 'conviction', 'zhangs_metric', 'jaccard', 'certainty', 'kulczynski']
rules_uf = rules_uf_pista.drop(columns=colunas_para_remover, axis=1)
rules_uf = rules_uf.sort_values(by='confidence', ascending=False)

In [9]:
rules_uf.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift
28,(uf_MS),(pista_Simples),0.035414,0.549288,0.030037,0.848188,1.544159
0,(uf_DF),(pista_Dupla),0.014640,0.366370,0.012216,0.834447,2.277606
20,(uf_BA),(pista_Simples),0.057822,0.549288,0.046672,0.807162,1.469468
40,(uf_TO),(pista_Simples),0.018759,0.549288,0.014947,0.796748,1.450509
32,(uf_PA),(pista_Simples),0.027707,0.549288,0.022075,0.796745,1.450504


In [23]:
# Ordena regras por confiança e pega top 5
top_5_confidence = rules_uf.sort_values(by='confidence', ascending=False).head(5)

print("Top 5 associações\n")

for _, row in top_5_confidence.iterrows():
    antecedents = ', '.join(list(row['antecedents']))
    consequents = ', '.join(list(row['consequents']))

    support_pct = row['support'] * 100
    confidence_pct = row['confidence'] * 100
    lift_value = row['lift']

    print(f"Acidentes que ocorreram em '{antecedents}' também ocorreram em '{consequents}', com {confidence_pct:.2f}% de confiança.")
    print()  

Top 5 associações

Acidentes que ocorreram em 'uf_MS' também ocorreram em 'pista_Simples', com 84.82% de confiança.

Acidentes que ocorreram em 'uf_DF' também ocorreram em 'pista_Dupla', com 83.44% de confiança.

Acidentes que ocorreram em 'uf_BA' também ocorreram em 'pista_Simples', com 80.72% de confiança.

Acidentes que ocorreram em 'uf_TO' também ocorreram em 'pista_Simples', com 79.67% de confiança.

Acidentes que ocorreram em 'uf_PA' também ocorreram em 'pista_Simples', com 79.67% de confiança.



Regra de associação considerando:
- `Condição meteorológica`
- `Fase do dia`

In [11]:
valores_unicos_condicao_meteorologica = df_acidentes['condicao_metereologica'].unique()
valores_unicos_fase_dia = df_acidentes['fase_dia'].unique()

print("Valores únicos na coluna 'condição meteorologica':")
print(valores_unicos_condicao_meteorologica)
print("\nValores únicos na coluna 'fase dia':")
print(valores_unicos_fase_dia)

Valores únicos na coluna 'condição meteorologica':
['Céu Claro' 'Nublado' 'Chuva' 'Garoa/Chuvisco' 'Sol' 'Ignorado'
 'Nevoeiro/Neblina' 'Vento' 'Neve' 'Granizo']

Valores únicos na coluna 'fase dia':
['Plena Noite' 'Amanhecer' 'Pleno dia' 'Anoitecer']


In [12]:
# Preparar as transações para Condição Meteorológica e Fase do Dia
transactions_condicao_fase = []
for index, row in df_acidentes.iterrows():
    transaction = []   
    transaction.append(f"condicao_{row['condicao_metereologica']}")
    transaction.append(f"fase_{row['fase_dia']}")
    transactions_condicao_fase.append(transaction)

# Converte para o formato adequado para o TransactionEncoder
condicao_fase = TransactionEncoder()
array_condicao_fase = condicao_fase.fit(transactions_condicao_fase).transform(transactions_condicao_fase)
df_encoded_condicao_fase = pd.DataFrame(array_condicao_fase, columns=condicao_fase.columns_)

In [13]:
# Aplicando Apriori com min_support 1% e a regra de associação
frequent_itemsets_condicao_fase = apriori(df_encoded_condicao_fase, min_support=0.01, use_colnames=True)
rules_condicao_fase = association_rules(frequent_itemsets_condicao_fase, metric="lift", min_threshold=1.0)

In [14]:
rules_condicao = rules_condicao_fase.drop(columns=colunas_para_remover, axis=1)
rules_condicao = rules_condicao.sort_values(by='confidence', ascending=False)

In [15]:
rules_condicao.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift
9,(condicao_Sol),(fase_Pleno dia),0.053919,0.541797,0.052510,0.973866,1.797475
2,(fase_Plena Noite),(condicao_Céu Claro),0.349008,0.620343,0.221884,0.635757,1.024848
5,(condicao_Garoa/Chuvisco),(fase_Plena Noite),0.043752,0.349008,0.019666,0.449492,1.287913
7,(condicao_Nublado),(fase_Plena Noite),0.145394,0.349008,0.054830,0.377109,1.080517
3,(condicao_Céu Claro),(fase_Plena Noite),0.620343,0.349008,0.221884,0.357680,1.024848


In [24]:
# Ordena regras por confiança e pega top 5
top_5_confidence = rules_condicao.sort_values(by='confidence', ascending=False).head(5)

print("Top 5 associações\n")

for _, row in top_5_confidence.iterrows():
    antecedents = ', '.join(list(row['antecedents']))
    consequents = ', '.join(list(row['consequents']))

    support_pct = row['support'] * 100
    confidence_pct = row['confidence'] * 100
    lift_value = row['lift']

    print(f"Acidentes que ocorreram com '{antecedents}' também ocorreram em '{consequents}', com {confidence_pct:.2f}% de confiança.")
    print()  

Top 5 associações

Acidentes que ocorreram com 'condicao_Sol' também ocorreram em 'fase_Pleno dia', com 97.39% de confiança.

Acidentes que ocorreram com 'fase_Plena Noite' também ocorreram em 'condicao_Céu Claro', com 63.58% de confiança.

Acidentes que ocorreram com 'condicao_Garoa/Chuvisco' também ocorreram em 'fase_Plena Noite', com 44.95% de confiança.

Acidentes que ocorreram com 'condicao_Nublado' também ocorreram em 'fase_Plena Noite', com 37.71% de confiança.

Acidentes que ocorreram com 'condicao_Céu Claro' também ocorreram em 'fase_Plena Noite', com 35.77% de confiança.



Regra de associação considerando:
- `Tipos de pista`
-  `Acidentes com ferimentos graves e morte`

In [29]:
# Renomeando colunas
df_acidentes = df_acidentes.rename(columns={"tem_ferido_grave": "ferimentos_graves", "tem_morto": "óbitos"})

df_acidentes['ferimentos_graves'] = df_acidentes['feridos_graves'] > 0
df_acidentes['óbitos'] = df_acidentes['mortos'] > 0

pistas = pd.get_dummies(df_acidentes['tipo_pista'], prefix='pista')
df_apriori = pd.concat([pistas, df_acidentes[['ferimentos_graves', 'óbitos']]], axis=1).astype(bool)

frequentes = apriori(df_apriori, min_support=0.01, use_colnames=True)
regras = association_rules(frequentes, metric="lift", min_threshold=0.8)

regras_pista = regras[regras['antecedents'].apply(lambda x: any(item.startswith('pista_') for item in x))]
regras_pista[['antecedents', 'consequents', 'support', 'confidence', 'lift']]


,antecedents,consequents,support,confidence,lift
1,(pista_Dupla),(ferimentos_graves),0.036002,0.098267,0.852918
3,(pista_Simples),(ferimentos_graves),0.071651,0.130443,1.132197
5,(pista_Simples),(óbitos),0.036660,0.066741,1.350532


In [33]:
# Ordena regras por confiança e pega top 5
top_5_confidence = regras_pista.sort_values(by='confidence', ascending=False).head(5)

print("Top 5 associações\n")

for _, row in top_5_confidence.iterrows():
    antecedents = ', '.join(list(row['antecedents']))
    consequents = ', '.join(list(row['consequents']))

    support_pct = row['support'] * 100
    confidence_pct = row['confidence'] * 100
    lift_value = row['lift']

    print(f"Acidentes que ocorreram em '{antecedents}' também apresentaram '{consequents}', com {confidence_pct:.2f}% de confiança.")
    print()  

Top 5 associações

Acidentes que ocorreram em 'pista_Simples' também apresentaram 'ferimentos_graves', com 13.04% de confiança.

Acidentes que ocorreram em 'pista_Dupla' também apresentaram 'ferimentos_graves', com 9.83% de confiança.

Acidentes que ocorreram em 'pista_Simples' também apresentaram 'óbitos', com 6.67% de confiança.



#### Considerações Finais

___

Nesta etapa foi aplicado mineração por regras de associação (Apriori) a pares de variáveis  como (Estado, tipo de pista, traçado, condição meteorológica e fase do dia) com objetivo de identificar co‑ocorrências relevantes em acidentes rodoviários. 

Os principais achados mostram que certas UFs (por exemplo MS, BA, TO, PA) apresentam alta probabilidade de ocorrer em pistas do tipo 'Simples' (confiança entre ≈79% e 85%; lift entre ≈1.45 e 1.54), enquanto o DF se destaca por sua forte associação com pista 'Dupla' (confiança ≈83%; lift ≈2.28). 

Esses resultados fornecem pistas úteis para priorização de ações: trechos de pista simples devem ser avaliados quanto à infraestrutura e medidas de mitigação da gravidade (melhoria do acostamento, iluminação e dispositivos de contenção), e políticas específicas por estado podem ser mais eficientes do que intervenções generalistas. 

Para trabalhos futuros, é possível realizar uma análise estatística mais aprofundada que teste até que ponto as correlações identificadas podem indicar causalidade.